In [1]:
import cv2
import numpy as np
import os
import pyautogui
import time
import mediapipe as mp
import vlc



In [2]:
def finger_raised(lst):
    fingers_raised_count = 0

    thresh = (lst.landmark[0].y * 100 - lst.landmark[9].y * 100) / 2

    if (lst.landmark[5].y * 100 - lst.landmark[8].y * 100) > thresh and (lst.landmark[9].y * 100 < lst.landmark[12].y * 100) and (lst.landmark[13].y * 100 < lst.landmark[16].y * 100) and (lst.landmark[17].y * 100 < lst.landmark[20].y * 100):     # Ring and Pinky fingers folded
        fingers_raised_count += 1

    # Middle, ring, and pinky fingers raised while index finger is folded
    if (lst.landmark[9].y * 100 - lst.landmark[12].y * 100) > thresh \
            and (lst.landmark[13].y * 100 - lst.landmark[16].y * 100) > thresh \
            and (lst.landmark[17].y * 100 - lst.landmark[20].y * 100) > thresh \
            and (lst.landmark[5].y * 100 < lst.landmark[8].y * 100) \
            and (lst.landmark[4].y * 100 < lst.landmark[8].y * 100):
        fingers_raised_count += 2

    # Ring and pinky fingers raised while middle finger is folded
    if (lst.landmark[13].y * 100 - lst.landmark[16].y * 100) > thresh \
            and (lst.landmark[17].y * 100 - lst.landmark[20].y * 100) > thresh \
            and (lst.landmark[9].y * 100 < lst.landmark[12].y * 100):
        fingers_raised_count += 3

    # Pinky finger raised while ring finger is folded
    if (lst.landmark[17].y * 100 - lst.landmark[20].y * 100) > thresh \
            and (lst.landmark[13].y * 100 < lst.landmark[16].y * 100):
        fingers_raised_count += 4

    # All the fingers raised
    if (lst.landmark[5].y * 100 - lst.landmark[8].y * 100) > thresh \
        and (lst.landmark[9].y * 100 - lst.landmark[12].y * 100) > thresh \
        and (lst.landmark[13].y * 100 - lst.landmark[16].y * 100) > thresh \
        and (lst.landmark[17].y * 100 - lst.landmark[20].y * 100) > thresh:
        fingers_raised_count += 6

    # All the fingers folded
    if (lst.landmark[5].y * 100 < lst.landmark[8].y * 100) \
        and (lst.landmark[9].y * 100 < lst.landmark[12].y * 100) \
        and (lst.landmark[13].y * 100 < lst.landmark[16].y * 100) \
        and (lst.landmark[17].y * 100 < lst.landmark[20].y * 100) \
        and (lst.landmark[4].y * 100 < lst.landmark[8].y * 100):
        fingers_raised_count += 7

    # # All fingers down 
    # if ((lst.landmark[5].y * 100 * 100 > lst.landmark[8].y * 100 * 100) and 
    #     (lst.landmark[9].y * 100 * 100 > lst.landmark[12].y * 100 * 100) and
    #     (lst.landmark[13].y * 100 * 100 > lst.landmark[16].y * 100 * 100) and
    #     (lst.landmark[17].y * 100 * 100 > lst.landmark[20].y * 100 * 100) and
    #     not (lst.landmark[5].x * 100 - lst.landmark[4].x * 100) > 6):
    #     fingers_raised_count += 6

    # # All fingers raise       
    # if ((lst.landmark[5].y * 100 * 100 < lst.landmark[8].y * 100 * 100) and 
    #     (lst.landmark[9].y * 100 * 100 < lst.landmark[12].y * 100 * 100) and
    #     (lst.landmark[13].y * 100 * 100 < lst.landmark[16].y * 100 * 100) and
    #     (lst.landmark[17].y * 100 * 100 < lst.landmark[20].y * 100 * 100) and
    #     (lst.landmark[5].x * 100 - lst.landmark[4].x * 100) > 6):
    #     fingers_raised_count += 7

    return fingers_raised_count

This function bellow gives positions(numbers) to ever joint on the hand 

In [3]:
# Create instance of VLC player
vlc_instance = vlc.Instance('--input-repeat=-1', '--fullscreen')
player = vlc_instance.media_player_new()
media_file_path = 'Ore dake Level Up na KenEP.7.5.v1.1080p.mp4'  # Replace with your video file path
media = vlc_instance.media_new(media_file_path)
player.set_media(media)
player.play()

# Your finger_raised function remains the same

# Initialize MediaPipe hands module
hand = mp.solutions.hands
handobj = hand.Hands(max_num_hands=1)
drawings = mp.solutions.drawing_utils

In [4]:
# Initialize camera
cap = cv2.VideoCapture(0)
Camwidth = 2000
Camheight = 2000
cap.set(3, Camwidth)
cap.set(4, Camheight)

previousTime = 0
prevt = -1
start_init = False

while True:
    ending_time = time.time()
    successful, imgg = cap.read()
    
    imgg = cv2.flip(imgg, 1)
    
    ress = handobj.process(cv2.cvtColor(imgg, cv2.COLOR_BGR2RGB))
    
    if ress.multi_hand_landmarks:
        
        the_hand_keypoints = ress.multi_hand_landmarks[0]
        
        fingers_raised_count = finger_raised(the_hand_keypoints)
        
        if not(prevt == fingers_raised_count):
            print(prevt)
            if not(start_init):
                starting_time = time.time()
                start_init = True
            elif (ending_time - starting_time > 0.2):  # Changed delay to 1 second
                if fingers_raised_count == 1:
                    if player.is_playing():
                        player.pause()
                        print("Video paused")
                    else:
                        player.play()
                        print("Video resumed")
                elif fingers_raised_count == 2:
                    player.set_time(player.get_time() + 20000)  # Fast forward by 20 seconds (in milliseconds)
                    print("Fast-forwarded by 20 seconds")
                elif fingers_raised_count == 3:
                    player.set_time(player.get_time() - 20000)  # Rewind by 20 seconds (in milliseconds)
                    print("Rewinded by 20 seconds")
                elif fingers_raised_count == 4:
                    current_volume = player.audio_get_volume()
                    player.audio_set_volume(max(0, current_volume - 30))  # Reduce volume by 30
                    print("Volume reduced by 30")
                elif fingers_raised_count == 6:
                    current_volume = player.audio_get_volume()
                    player.audio_set_volume(min(100, current_volume + 30))  # Increase volume by 30
                    print("Volume increased by 30")

                prevt = fingers_raised_count
                start_init = False
        
        prevt = fingers_raised_count
        
        drawings.draw_landmarks(imgg, the_hand_keypoints, hand.HAND_CONNECTIONS)
    
    cureentTime = time.time()

    fps = 1 / (cureentTime - previousTime)

    previousTime = cureentTime
    
    cv2.putText(imgg, f'FPS: {int(fps)}', (10, 40),
                cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

    cv2.imshow("Image", imgg)
    cv2.waitKey(2)

    cv2.putText(imgg, f'FPS: {int(fps)}', (10, 40),
                cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

    cv2.imshow("Image", imgg)
    cv2.waitKey(2)


-1
6
